In [1]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(".."))

import src.preprocessing
from importlib import reload
reload(src.preprocessing)

from typing import Union
from src.preprocessing import compute_cancellation, aggregate_user_day_activity, add_days_since, add_rolling_averages

In [2]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df = pd.read_parquet(root + '/data/train.parquet')
df.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [3]:
df_new = aggregate_user_day_activity(df)
df_new  = add_days_since(df_new)
df_new = add_rolling_averages(df_new, columns=['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up'], n=30)
df_new = add_rolling_averages(df_new, columns=['Thumbs Down', 'Thumbs Up'], n=7)

In [ ]:
# Compute cancellation target for each date in df_new
# For each unique date, compute which users cancelled within 10 days

unique_dates = sorted(df_new['date'].unique())
print(f"Computing cancellation targets for {len(unique_dates)} unique dates...")

cancellation_targets = []

for present_date in unique_dates:
    # Compute cancellation for this date
    target_df = compute_cancellation(df, present_time=present_date, window_days=10)
    # Add the date column
    target_df['date'] = present_date
    cancellation_targets.append(target_df)

# Combine all targets
target_by_date = pd.concat(cancellation_targets, ignore_index=True)

# Rename columns for clarity
target_by_date = target_by_date.rename(columns={'userId': 'userId', 'will_churn': 'churn_status'})

print(f"\nCancellation targets shape: {target_by_date.shape}")
print(f"Sample:")
print(target_by_date.head(10))

Computing cancellation targets for 51 unique dates...

Cancellation targets shape: (976140, 3)
Sample:
    userId  will_churn        date
0  1749042           0  2018-10-01
1  1563081           0  2018-10-01
2  1697168           0  2018-10-01
3  1222580           0  2018-10-01
4  1714398           0  2018-10-01
5  1010522           0  2018-10-01
6  1475659           0  2018-10-01
7  1558463           0  2018-10-01
8  1605667           0  2018-10-01
9  1385500           0  2018-10-01

Cancellation targets shape: (976140, 3)
Sample:
    userId  will_churn        date
0  1749042           0  2018-10-01
1  1563081           0  2018-10-01
2  1697168           0  2018-10-01
3  1222580           0  2018-10-01
4  1714398           0  2018-10-01
5  1010522           0  2018-10-01
6  1475659           0  2018-10-01
7  1558463           0  2018-10-01
8  1605667           0  2018-10-01
9  1385500           0  2018-10-01


In [18]:
# Save df_new and target_by_date as CSV files
df_new.to_csv(root + '/data/df_transformed.csv', index=False)
target_by_date.to_csv(root + '/data/churn_status.csv', index=False)

print(f"Saved df_new to data/df_transformed.csv ({df_new.shape})")
print(f"Saved target_by_date to data/churn_status.csv ({target_by_date.shape})")

Saved df_new to data/df_transformed.csv ((622838, 29))
Saved target_by_date to data/churn_status.csv ((976140, 3))


In [19]:
# Merge df_new with target_by_date to create training dataset
df_train = df_new.merge(target_by_date, on=['userId', 'date'], how='left')

print(f"df_train shape: {df_train.shape}")
print(f"Columns: {df_train.columns.tolist()}")
print(f"\nChurn distribution:")
print(df_train['churn_status'].value_counts())
print(f"\nSample:")
print(df_train.head())

df_train shape: (622838, 30)
Columns: ['date', 'userId', 'About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade', 'days_since_submit_downgrade', 'days_since_submit_upgrade', 'days_since_cancel', 'add_friend_avg_30d', 'add_to_playlist_avg_30d', 'thumbs_down_avg_30d', 'thumbs_up_avg_30d', 'thumbs_down_avg_7d', 'thumbs_up_avg_7d', 'churn_status']

Churn distribution:
churn_status
0    587760
1     35078
Name: count, dtype: int64

Sample:
         date   userId  About  Add Friend  Add to Playlist  Cancel  Downgrade  \
0  2018-10-02  1000025      0           3                2       0          1   
1  2018-10-03  1000025      0           6               13       0          2   
2  2018-10-04  1000025      0           0                5       0          3   
3  2018-10-05  1000025      0           0                1   

In [20]:
# Train a Random Forest model to predict churn_status
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Prepare data: drop rows with NaN in target
df_model = df_train.dropna(subset=['churn_status']).copy()
print(f"Training data shape after dropping NaN targets: {df_model.shape}")

# Separate features and target
# Exclude userId, date, and churn_status from features
exclude_cols = ['userId', 'date', 'churn_status']
feature_cols = [col for col in df_model.columns if col not in exclude_cols]

# Handle remaining NaN values in features (fill with 0 or median)
X = df_model[feature_cols].fillna(0)
y = df_model['churn_status'].astype(int)

print(f"\nFeatures: {len(feature_cols)} columns")
print(f"Feature columns: {feature_cols}")
print(f"\nTarget distribution:")
print(y.value_counts())

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTrain set: {X_train.shape}, Test set: {X_test.shape}")

# Train Random Forest
print("\nTraining Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("\n=== Model Performance ===")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Training data shape after dropping NaN targets: (622838, 30)


/var/folders/69/x1vfxvpd1_q07my0mb9rc8tm0000gn/T/ipykernel_3092/629828143.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = df_model[feature_cols].fillna(0)



Features: 27 columns
Feature columns: ['About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade', 'days_since_submit_downgrade', 'days_since_submit_upgrade', 'days_since_cancel', 'add_friend_avg_30d', 'add_to_playlist_avg_30d', 'thumbs_down_avg_30d', 'thumbs_up_avg_30d', 'thumbs_down_avg_7d', 'thumbs_up_avg_7d']

Target distribution:
churn_status
0    587760
1     35078
Name: count, dtype: int64

Train set: (498270, 27), Test set: (124568, 27)

Training Random Forest...

Train set: (498270, 27), Test set: (124568, 27)

Training Random Forest...

=== Model Performance ===
ROC-AUC Score: 0.7257

Confusion Matrix:
[[117552      0]
 [  6150    866]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    117552
           1       1.00      0.12     

In [22]:
# Load and transform test data with the same pipeline as training data
print("Loading test data...")
df_test = pd.read_parquet(root + '/data/test.parquet')
print(f"Test data shape: {df_test.shape}")
print(f"Date range: {df_test['time'].min()} to {df_test['time'].max()}")

# Apply the same transformations
print("\nApplying transformations to test data...")

# 1. Aggregate by user and day
df_test_agg = aggregate_user_day_activity(df_test, fill_missing_days=True)
print(f"After aggregation: {df_test_agg.shape}")
print(f"Available columns: {df_test_agg.columns.tolist()}")

# 2. Add days since metrics - only track columns that exist in test data
available_tracking_cols = []
for col in ['Submit Downgrade', 'Submit Upgrade', 'Cancel']:
    if col in df_test_agg.columns:
        available_tracking_cols.append(col)

if available_tracking_cols:
    df_test_agg = add_days_since(df_test_agg, columns_to_track=available_tracking_cols)
    print(f"After days_since: {df_test_agg.shape}")
else:
    print("No tracking columns found, skipping days_since")

# 3. Add 30-day rolling averages
activity_cols_30 = ['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up']
available_cols_30 = [col for col in activity_cols_30 if col in df_test_agg.columns]
if available_cols_30:
    df_test_agg = add_rolling_averages(df_test_agg, columns=available_cols_30, n=30)
    print(f"After 30d rolling averages: {df_test_agg.shape}")

# 4. Add 7-day rolling averages
activity_cols_7 = ['Thumbs Down', 'Thumbs Up']
available_cols_7 = [col for col in activity_cols_7 if col in df_test_agg.columns]
if available_cols_7:
    df_test_agg = add_rolling_averages(df_test_agg, columns=available_cols_7, n=7)
    print(f"After 7d rolling averages: {df_test_agg.shape}")

print("\nTest data ready for prediction!")
print(f"Final test features shape: {df_test_agg.shape}")
print(f"Columns: {df_test_agg.columns.tolist()}")

Loading test data...
Test data shape: (4393179, 19)
Date range: 2018-10-01 00:00:06 to 2018-11-20 00:00:00

Applying transformations to test data...
Test data shape: (4393179, 19)
Date range: 2018-10-01 00:00:06 to 2018-11-20 00:00:00

Applying transformations to test data...
After aggregation: (117479, 22)
Available columns: ['date', 'userId', 'About', 'Add Friend', 'Add to Playlist', 'Downgrade', 'Error', 'Help', 'Home', 'Login', 'Logout', 'NextSong', 'Register', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Registration', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade']
After aggregation: (117479, 22)
Available columns: ['date', 'userId', 'About', 'Add Friend', 'Add to Playlist', 'Downgrade', 'Error', 'Help', 'Home', 'Login', 'Logout', 'NextSong', 'Register', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Registration', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade']
After days_since: (117479, 24)
After days_since: 

In [25]:
# Make predictions on test data using only the last date
print("Preparing test data for model prediction...")

# Get the last date in test data
last_date = df_test_agg['date'].max()
print(f"Last date in test data: {last_date}")

# Filter to only predictions for the last date
df_test_final = df_test_agg[df_test_agg['date'] == last_date].copy()
print(f"Number of users on last date: {len(df_test_final)}")

# Get the exact feature columns used in training (from feature_cols variable)
print(f"\nTraining features: {feature_cols}")

# Add missing columns from training to test data with zeros
for col in feature_cols:
    if col not in df_test_final.columns:
        print(f"Adding missing column: {col}")
        df_test_final[col] = 0

# Select features in the same order as training
X_test_pred = df_test_final[feature_cols].fillna(0)

print(f"\nUsing {len(feature_cols)} features for prediction")
print(f"Test data shape for prediction: {X_test_pred.shape}")

# Make predictions
print("Making predictions...")
test_predictions = rf_model.predict(X_test_pred)
test_predictions_proba = rf_model.predict_proba(X_test_pred)[:, 1]

print(f"Predictions shape: {test_predictions.shape}")
print(f"Prediction distribution:")
print(pd.Series(test_predictions).value_counts())

# Create submission dataframe with userId as id and predictions as target
submission = pd.DataFrame({
    'id': df_test_final['userId'].values,
    'target': test_predictions
})

print(f"\nSubmission dataframe shape: {submission.shape}")
print(f"Sample submissions:")
print(submission.head(10))

# Save submission
submission.to_csv(root + '/data/submission.csv', index=False)
print(f"\nSaved submission to data/submission.csv")
print(f"Submission target distribution:")
print(submission['target'].value_counts())

Preparing test data for model prediction...
Last date in test data: 2018-11-20
Number of users on last date: 2

Training features: ['About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade', 'days_since_submit_downgrade', 'days_since_submit_upgrade', 'days_since_cancel', 'add_friend_avg_30d', 'add_to_playlist_avg_30d', 'thumbs_down_avg_30d', 'thumbs_up_avg_30d', 'thumbs_down_avg_7d', 'thumbs_up_avg_7d']

Using 27 features for prediction
Test data shape for prediction: (2, 27)
Making predictions...
Predictions shape: (2,)
Prediction distribution:
0    2
Name: count, dtype: int64

Submission dataframe shape: (2, 2)
Sample submissions:
        id  target
0  1270265       0
1  1835314       0

Saved submission to data/submission.csv
Submission target distribution:
target
0    2
Name: count, dtype: int64


/var/folders/69/x1vfxvpd1_q07my0mb9rc8tm0000gn/T/ipykernel_3092/232987868.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_pred = df_test_final[feature_cols].fillna(0)


In [ ]:
# Check how many users have observations on the last date in the raw test data
last_date_pd = pd.to_datetime(last_date)

# Convert time column to datetime for comparison
df_test['time_date'] = pd.to_datetime(df_test['time']).dt.date

# Count users on the last date
users_on_last_date = df_test[df_test['time_date'] == last_date]['userId'].nunique()
total_events_on_last_date = len(df_test[df_test['time_date'] == last_date])

print(f"Last date: {last_date}")
print(f"Unique users with events on {last_date}: {users_on_last_date}")
print(f"Total events on {last_date}: {total_events_on_last_date}")

# Also check the date range in raw test data
print(f"\nDate range in raw test data:")
print(f"First date: {df_test['time_date'].min()}")
print(f"Last date: {df_test['time_date'].max()}")
print(f"\nTotal unique dates in test data: {df_test['time_date'].nunique()}")
print(f"Total unique users in test data: {df_test['userId'].nunique()}")